<div style="text-align: center;">
  <h1>Analisis de sentimiento de reseñas</h1>
</div>

In [1]:
import pandas as pd
import nltk # Importamos nuestro bello nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer # Este es el modelo que decidi utilizar
from nltk.corpus import stopwords # Importamos las stopwords
import re # Importamos re
nltk.download('punkt') # Instalamos punkt
nltk.download('stopwords') # Instalamos las stopwords
nltk.download('vader_lexicon') # Y instalamos nuestro vader de analisis

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
df_reviews = pd.read_parquet(r'data/clean_reviews.parquet.gzip') # Leemos nuestras reseñas

In [3]:
# Definir la función de tokenización y eliminacion de stopwords
def tokenizacion(x):
    if isinstance(x, str):
        # Stopwords
        stopwords_list = stopwords.words('english')
        stopwords_list = [word for word in stopwords_list if 'not' not in word]

        # Tokenización y eliminación de stopwords
        tokens = nltk.tokenize.word_tokenize(x)
        tokens = [word for word in tokens if word.lower() not in stopwords_list]

        return ' '.join(tokens)
        #return tokens
    else:
        return x  # Devuelve tal cual si no es una cadena

In [4]:
df_reviews['review'] = df_reviews['review'].apply(lambda x: tokenizacion(x)) # Aplicamos la funcion a nuestra columna reviews

In [5]:
# Creamos una funcion que elimina caracteres repetidos, ya que encontre
# reseñas tipo: "Nooooooooooooooooo" y las transforme a "Noo"

def eliminar_caracteres_repetidos(text):
    # Definir una expresión regular para encontrar caracteres repetidos más de 2 veces con re
    pattern = re.compile(r'(.)\1{2,}', re.DOTALL)
    # Reemplazar caracteres repetidos con dos instancias del mismo caracter
    text_limpio = pattern.sub(r'\1\1', text)
    return text_limpio
df_reviews['review'] = df_reviews['review'].apply(lambda x: eliminar_caracteres_repetidos(x)) # Aplicamos nuestra funcion de transformacion

In [6]:
# Instanciamos el SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

# Función para realizar análisis de sentimiento de las reseñas y asignar valor según escala.
# 0: Negativa ; 1: Neutra ; 2: Positiva

def analyze_sentiment(text):
    if pd.isnull(text) or text == '':
        return 1 # Si la reseña es nula se toma como neutra
    p_scores = sid.polarity_scores(text)
    sentiment = p_scores.get('compound') # Utilizamos el compound en vez de los valores independientes
    if sentiment > 0:
        return 2
    if sentiment < 0:
        return 0
    else:
        return 1


# Aplicar la función a la columna 'review' y cree la columna 'sentiment_analysis'
df_reviews['sentiment_analysis'] = df_reviews['review'].apply(analyze_sentiment)


In [7]:
df_reviews[['review', 'sentiment_analysis']].head(5) # Mostramos un ejemplo de nuestras reseñas
                                                     # Para ver como trabajo el modelo

,review,sentiment_analysis
0,simple yet great replayability opinion zombie ...,2
1,unique worth playthrough,2
2,great atmosphere gunplay bit chunky times end ...,2
3,know think see title barbie dreamhouse party n...,2
4,simple actually not simple truck driving simul...,2


In [8]:
df_reviews[df_reviews['sentiment_analysis']==0][['review', 'sentiment_analysis']]

# Aca decidi visualizar las reseñas negativas para ver que tan preciso era el modelo

,review,sentiment_analysis
22,reminds tv show called walking dead,0
25,nt get game not like greatest fps time anythin...,0
26,killed emperor nobody cared got away accidenta...,0
38,compared left dead game really gives creeps ad...,0
39,wo nt regret playing,0
...,...,...
48461,death great teacher failure counterrecommended...,0
48463,portal great game created valve basically guin...,0
48471,lost skins getting hacked rip skins gaben giff...,0
48478,simply horrible always lag always gets frozen ...,0


Y me parecio que era muy preciso

In [9]:
df_reviews.to_parquet(r'data/clean_reviews.parquet.gzip', compression='gzip') # Lo exportamos como el mismo archivo